In [27]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from collections import OrderedDict

In [5]:
# Download the dataset
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)

mnist_data = datasets.MNIST(
    root="../data/raw/",
    train=True,
    transform=transforms.ToTensor(),
    download=True,
)
dataloader = DataLoader(mnist_data, batch_size=64, shuffle=True)

6.6%

100.0%


Extracting ../data/raw/MNIST/raw/train-images-idx3-ubyte.gz to ../data/raw/MNIST/raw



100.0%


Extracting ../data/raw/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/raw/MNIST/raw



51.7%

100.0%


Extracting ../data/raw/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/raw/MNIST/raw



100.0%

Extracting ../data/raw/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/raw/MNIST/raw



In [8]:
# check the quantity of the dataset
total_data = 0

for data, label in dataloader:
    total_data+=data.shape[0]

print("The total quantity of the dataset # {}".format(total_data))
print("The shape of the dataset # {} ".format(data.shape))

The total quantity of the dataset # 60000
The shape of the dataset # torch.Size([32, 1, 28, 28]) 


In [9]:
# Call the GPU - MAC
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

print("Device # {} ".format(device))

Device # mps 


In [50]:
class Generator(nn.Module):
    """
    A generator class for a Generative Adversarial Network (GAN), particularly used for
    generating images. It takes a latent space vector and a label as input and generates
    images corresponding to the input label. It utilizes fully connected layers and
    LeakyReLU activation for intermediate layers, with a Tanh activation for the output layer.

    Attributes:
        latent_space (int): Dimensionality of the latent space vector (z), which is a random
                            noise input for the generator.
        num_labels (int): Number of unique labels for the conditional GAN. It corresponds to
                          the number of different classes in the dataset.
        labels (nn.Embedding): Embedding layer for the labels, allowing the generator to use
                               label information to generate images corresponding to specific classes.
        layers_config (list): A list defining the architecture of the neural network. Each
                              element in the list is a tuple, with the first two elements
                              being the number of input and output features for a layer,
                              and the optional third being the negative slope for LeakyReLU.
        model (nn.Sequential): The actual neural network model, constructed based on layers_config.
                               It comprises fully connected (Linear) layers, LeakyReLU activation
                               for non-linearity in intermediate layers, and a Tanh activation function
                               in the output layer for generating pixel values.

    Methods:
        connected_layer(layers_config=None):
            Constructs the neural network layers based on layers_config. It initializes the fully
            connected layers and the activation functions, specifically using LeakyReLU for intermediate
            layers and Tanh for the output layer.

        forward(x, labels):
            Performs a forward pass of the generator. It takes a latent space vector `x` and its
            corresponding labels, processes them through the network, and generates a batch of images.

    Note:
        - The latent space vector x should be of the shape (N, latent_space) where N is the batch size.
        - Labels should be of shape (N,) and contain integers representing class labels.
        - The output is a tensor of shape (N, 1, 28, 28), representing generated images of size 28x28 pixels.
    """

    def __init__(self, latent_space=100, num_labels=10):
        self.latent_space = latent_space
        self.num_labels = num_labels
        super(Generator, self).__init__()
        self.labels = nn.Embedding(self.num_labels, self.num_labels)
        self.layers_config = [
            (self.latent_space + self.num_labels, 256, 0.2),
            (256, 512, 0.2),
            (512, 1024, 0.2),
            (1024, 784),
        ]
        self.model = self.connected_layer(layers_config=self.layers_config)

    def connected_layer(self, layers_config=None):
        layers = OrderedDict()
        if layers_config is not None:
            for index, (in_features, out_features, negative_slope) in enumerate(
                layers_config[:-1]
            ):
                layers["{}_layer".format(index)] = nn.Linear(
                    in_features=in_features, out_features=out_features
                )
                layers["{}_activation".format(index)] = nn.LeakyReLU(
                    negative_slope=negative_slope
                )

            (in_features, out_features) = layers_config[-1]
            layers["output_layer"] = nn.Linear(
                in_features=in_features, out_features=out_features
            )
            layers["output_activation"] = nn.Tanh()

            return nn.Sequential(layers)

        else:
            raise Exception("No layers config provided".capitalize())

    def forward(self, x, labels):
        if x is not None:
            labels = self.labels(labels)
            x = torch.cat([x, labels], dim=1)
            x = self.model(x)
        else:
            raise Exception("No input provided in Generator".capitalize())

        return x.reshape(-1, 1, 28, 28)

In [51]:
g = Generator()
noise_data = torch.randn(64, 100)
data, labels = next(iter(dataloader))

g(noise_data, labels).shape

torch.Size([64, 1, 28, 28])

In [40]:
class Discriminator(nn.Module):
    """
    A discriminator class for a Generative Adversarial Network (GAN), particularly used for
    image data with an added conditional label embedding. The discriminator's goal is to
    differentiate between real and fake images. It utilizes fully connected layers,
    LeakyReLU activation for hidden layers, and a Sigmoid activation for the output layer.

    Attributes:
        num_labels (int): Number of unique labels for the conditional GAN. For instance,
                          in a dataset with 10 different classes, num_labels should be 10.
        labels (nn.Embedding): Embedding layer for the labels, which allows the discriminator
                               to condition the input on a particular class.
        layers_config (list): A list defining the architecture of the neural network. Each
                              element in the list is a tuple, with the first two elements
                              being the number of input and output features for a layer,
                              and the optional third being the negative slope for LeakyReLU.
        model (nn.Sequential): The actual neural network model, constructed based on layers_config.
                              It comprises fully connected (Linear) layers, LeakyReLU activation
                              for non-linearity in hidden layers, and a Sigmoid activation function
                              in the output layer to obtain probabilities.

    Methods:
        connected_layer(layers_config=None):
            Constructs the neural network layers based on layers_config. It initializes the fully
            connected layers and the activation functions, specifically using LeakyReLU for hidden
            layers and Sigmoid for the output layer.

        forward(x, labels):
            Performs a forward pass of the discriminator. It takes an input batch of images `x`
            and their corresponding labels, processes the images and labels through the network,
            and outputs a batch of probabilities indicating how likely each image is to be real.

    Note:
        - The model expects inputs x to be of the shape (N, 784) where N is the batch size.
        - Labels should be of shape (N,) and contain integers representing class labels.
        - The output is of shape (N, 1), representing the likelihood of each image being real.
        - The network architecture is designed to work with flattened images of size 28x28 pixels.
    """

    def __init__(self, num_labels=10):
        self.num_labels = num_labels
        super(Discriminator, self).__init__()
        self.labels = nn.Embedding(self.num_labels, self.num_labels)
        self.layers_config = [
            (784 + self.num_labels, 512, 0.2),
            (512, 256, 0.2),
            (256, 1),
        ]
        self.model = self.connected_layer(layers_config=self.layers_config)

    def connected_layer(self, layers_config=None):
        layers = OrderedDict()
        if layers_config is not None:
            for index, (in_features, out_features, negative_slope) in enumerate(
                layers_config[:-1]
            ):
                layers["{}_layer".format(index + 1)] = nn.Linear(
                    in_features=in_features, out_features=out_features
                )
                layers["{}_activation".format(index + 1)] = nn.LeakyReLU(
                    negative_slope=negative_slope
                )

            (in_features, out_features) = layers_config[-1]
            layers["output_layer"] = nn.Linear(
                in_features=in_features, out_features=out_features
            )
            layers["output_activation"] = nn.Sigmoid()

            return nn.Sequential(layers)
        else:
            raise Exception("Layers config is not defined properly".capitalize())

    def forward(self, x, labels):
        if x is not None:
            labels = self.labels(labels)
            x = x.reshape(-1, 28 * 28)
            x = torch.cat([x, labels], dim=1)
            x = self.model(x)
        else:
            raise Exception("Inputs are not defined properly".capitalize())

        return x

In [42]:
d = Discriminator()

data, label = next(iter(dataloader))
d(data, label).shape

torch.Size([64, 1])

In [41]:
d = Discriminator()

data = torch.randn(64, 1, 28, 28)

d(data, label).shape

torch.Size([64, 1])

In [ ]:
'''
    x = (64, 1, 28, 28)
    labels = (64, 10)
'''